In [ ]:
import requests
import pandas as pd

# Extended gene list
gene_list = [
    "TP53", "BRCA1", "BRCA2", "EGFR", "PIK3CA", "PTEN", "KRAS", "NRAS", "HRAS",
    "AKT1", "AKT2", "CDK1", "CDK2", "CDK4", "CDK6", "CCND1", "CCNE1", "RB1",
    "MYC", "MYCN", "MDM2", "ATM", "ATR", "CHEK1", "CHEK2", "BCL2", "BAX", "CASP3"
] # gene list 

url = "https://biit.cs.ut.ee/gprofiler/api/gost/profile/"

payload = {
    "organism": "hsapiens",
    "query": gene_list,
    "sources": ["GO:BP", "CORUM"], #change the source names as needed
    "user_threshold": 0.05,
    "domain_scope": "annotated"
}

response = requests.post(url, json=payload)

if response.status_code == 200:
    result = response.json()
    print("Enrichment successful. Found", len(result["result"]), "terms.")
    print("Top-level keys in result:", result.keys())  # Debug check

    gene_symbols = gene_list  # fallback if genes_metadata is not available

    # Add gene symbols for each enriched term
    for entry in result["result"]:
        entry["genes_with_evidence"] = [
            gene for gene, evidence in zip(gene_symbols, entry["intersections"]) if evidence
        ]

    # Create full DataFrame
    full_df = pd.DataFrame(result["result"])
    full_df["genes_with_evidence"] = full_df["genes_with_evidence"].apply(lambda x: ", ".join(x))

    # Select common columns
    selected_cols = ['source', 'name', 'native', 'significant', 'p_value', 'term_size',
                     'query_size', 'intersection_size', 'effective_domain_size', 'genes_with_evidence']

    # Filter into GO:BP and CORUM
    go_df = full_df[full_df["source"] == "GO:BP"][selected_cols]
    corum_df = full_df[full_df["source"] == "CORUM"][selected_cols]

    # Save each to its own file
    go_df.to_csv("go_bp_enrichment_results.csv", index=False)
    corum_df.to_csv("corum_enrichment_results.csv", index=False)

    print("GO:BP results saved to go_bp_enrichment_results.csv")
    print("CORUM results saved to corum_enrichment_results.csv")

else:
    print("API call failed with status code:", response.status_code)
    print("Error message:", response.text)


Enrichment successful. Found 467 terms.
Top-level keys in result: dict_keys(['result', 'meta'])
GO:BP results saved to go_bp_enrichment_results.csv
CORUM results saved to corum_enrichment_results.csv
